In [ ]:
import matplotlib.pyplot as plt
from pya import *

%matplotlib widget

# Simulation

In [ ]:
import simulation

n = 128
sim_speed = 0.004
sim_fps = 500
duration = 6
frame_amount = duration * sim_fps

initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.15) for x in range(n)] for y in range(n)])
# initial_state = np.array([[simulation.gaussian_impulse(x, y, n, offset=[-0.6, 0.0], width=0.15, impulse=0.1) for x in range(n)] for y in range(n)])
# initial_state = np.array([[simulation.gaussian(x, y, n, offset=[-0.6, 0.0], width=0.05) for x in range(n)] for y in range(n)]) + np.array([[simulation.gaussian(x, y, n, offset=[-0.62, 0.0], width=0.05, imag=True) for x in range(n)] for y in range(n)])
potential = np.array([[simulation.parabola(x, y, n, offset=[0, 0], factor=10000) for x in range(n)] for y in range(n)])
# potential = np.array([[complex(0, 0) for x in range(n)] for y in range(n)])

barrier_width = 2
multi_slit = [(-15, -13), (-8, -6), (-1, 1), (6, 8), (13, 15)]
double_slit = [(-8, -3), (3, 8)]
single_slit = [(-3, 3)]
slits = double_slit

# action
frames = simulation.sim(n, sim_fps, duration, slits, barrier_width, sim_speed, initial_state=initial_state, potential=potential)

In [ ]:
import matplotlib
plt.figure()
plt.pcolormesh(np.abs(frames[0]) ** 2, cmap='inferno', norm=matplotlib.colors.PowerNorm(vmin=0, vmax=np.max(np.square(np.abs(frames))), gamma=0.4))
# plt.pcolormesh(np.abs(potential), vmin=0, vmax=20000)
plt.colorbar()
plt.show()

# Video

In [ ]:
import video

# save video
video_filename, anim = video.create(frames, 20, frame_amount, sim_fps, slits, barrier_width, n, save=False)

# plt.close()